In [2]:
import pickle

In [7]:
def save_lists_to_file(path):
    # Open the pickle file in binary write mode
    with open(path, 'rb') as f:
    # Load each list from the file
        NE = pickle.load(f)
        EEG_segments = pickle.load(f)
        Classes = pickle.load(f)

    return NE, EEG_segments, Classes

In [8]:
#read in train and test data

train_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\train_NER.pkl"

test_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\test_NER.pkl"

In [9]:

train_NE, train_EEG_segments, train_Classes = save_lists_to_file(train_path)
test_NE, test_EEG_segments, test_Classes = save_lists_to_file(test_path)

In [11]:
train_Classes

[['B-PER', 'I-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-LOC', 'I-LOC'],
 ['B-PER', 'I-PER'],
 ['B-PER'],
 ['B-LOC'],
 ['B-LOC'],
 ['B-LOC'],
 ['B-PER', 'I-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER', 'I-PER'],
 ['B-ORG'],
 ['B-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER'],
 ['B-PER'],
 ['B-PER'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG'],
 ['B-ORG'],
 ['B-PER'],
 ['B-LOC'],
 ['B-LOC'],
 ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG'],
 ['B-ORG', 'I-ORG'],
 ['B-ORG', 'I-ORG'],
 ['B-ORG'],
 ['B-LOC', 'I-LOC'],
 ['B-LOC', 'I-LOC'],
 ['B-LOC'],
 ['B-LOC', 'B-ORG', 'I-ORG'],
 ['B-ORG', 'I-ORG', 'I-ORG'],
 ['B-LOC'],
 ['B-PER'],
 ['B-ORG', 'I-ORG'],
 ['B-PER', 'I-PER'],
 ['B-PER', 'I-PER', 'I-PER'],
 ['B-LOC'],
 ['B-LOC'],
 ['B-PER', 'I-PER'],
 ['B-PER', 'I-PER'],
 